# Specializētā derivatīvās morfoloģijas modeļa prototips

In [ ]:
# Autors: Ronalds Turnis
# Specializēta modeļa inicializācija, pamatapmācība un pielāgošana derivatīvās morfoloģijas vajadzībām latviešu valodā

# Importē nepieciešamās bibliotēkas
import os, math, random, optuna, csv, urllib.parse, requests
import torch, torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import PreTrainedTokenizer
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.amp import autocast, GradScaler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from optuna.exceptions import TrialPruned
from torch.utils.data import Subset
from peft import LoraConfig, get_peft_model, PeftModel

# TF32 režīms ātrākiem aprēķiniem
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

# Slēdži
run_optuna = False # Optuna hiperparametru algoritma slēdzis
pretrain = False # Pamatapmācības slēdzis
finetune = True # Pielāgošanas slēdzis
experiment = True # Eksperimenta izpildes slēdzis

# Fiksēta sēkla rezultātu atkārtojamībai
def set_seed(seed = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Tekstvienību apstrādes klase

In [2]:
class CustomCharTokenizer(PreTrainedTokenizer):
    def __init__(self, vocab, unk_token="[UNK]", pad_token="[PAD]", sos_token="[SOS]", eos_token="[EOS]"):
        object.__setattr__(self, "_vocab", vocab)
        object.__setattr__(self, "_vocab_size", len(vocab))

        # Inicializē speciālās tekstvienības
        super().__init__(unk_token=unk_token, pad_token=pad_token, sos_token=sos_token, eos_token=eos_token)

        # Izveido kartējumu "tekstvienības ID -> tekstvienība"
        self.ids_to_tokens = {i: token for token, i in vocab.items()}

        # Saglabā speciālās tekstvienības
        self.unk_token = unk_token
        self.pad_token = pad_token
        self.sos_token = sos_token
        self.eos_token = eos_token

    @property
    def vocab(self):
        return self._vocab

    @property
    def vocab_size(self):
        return self._vocab_size

    @property
    def pad_token_id(self):
        return self._vocab.get(self.pad_token)

    def get_vocab(self):
        return self._vocab

    def _tokenize(self, text):
        return list(text.lower()) # Ievades un izvades secības ir ar mazajiem alfabēta burtiem

    def _convert_token_to_id(self, token):
        return self._vocab.get(token, self._vocab.get(self.unk_token)) # Ja tekstvienības nav vārdnīcā, atgriež UNK

    def _convert_id_to_token(self, index):
        return self.ids_to_tokens.get(index, self.unk_token) # Atgriež tekstvienību pēc ID

    def convert_tokens_to_string(self, tokens):
        return "".join(tokens) # Savieno tekstvienības vienā teksta virknē

    def encode(self, text, max_length=32):
        # Pārvērš ievades secību uz tekstvienību secību
        tokens = self._tokenize(text)
        ids = [self._convert_token_to_id(t) for t in tokens]

        # Apgriež secību līdz 30 tekstvienībām
        body = ids[: max_length - 2]

        # Pievieno speciālās tekstvienības (kopā 32 tekstvienību secība)
        return [self._vocab[self.sos_token]] + body + [self._vocab[self.eos_token]]


    def decode(self, token_ids):
        # Atgriež tekstvienību secību no tekstvienību ID secības
        tokens = [self._convert_id_to_token(i) for i in token_ids]

        # Noņem SOS, EOS un PAD, lai netraucētu izvades secības lasāmībai
        skip = {self.sos_token, self.eos_token, self.pad_token}
        tokens = [t for t in tokens if t not in skip]

        # Atgriež tekstu
        return self.convert_tokens_to_string(tokens)

# Vārdnīcas definēšana
latvian_letters = list("aābcčdeēfgģhiījkķlļmnņoprsštuūvzž")
digits = list("0123456789")
additional_symbols = list("-")
special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"]

# Apvieno visas tekstvienības vienā sarakstā
vocab_tokens = special_tokens + latvian_letters + digits + additional_symbols

# Izveido vārdnīcu ar unikāliem indeksiem
vocab_dict = {token: idx for idx, token in enumerate(vocab_tokens)}

# Inicializē tekstvienību apstrādes instanci
tokenizer = CustomCharTokenizer(vocab_dict)

# Datu kopu apstrādes klase

In [3]:
class WordPairDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer, max_length = 32):
        # Pārvērš "Lemma" un "Word" kolonnās esošos datus par sarakstiem
        self.lemmas = dataframe["Lemma"].astype(str).tolist()
        self.words = dataframe["Word"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        del dataframe # Atbrīvo atmiņu no dataframe pēc apstrādes

    def __len__(self):
        return len(self.lemmas)

    def __getitem__(self, idx):
        # Pārveido saņemto pāri par tekstvienību secībām un atgriež kā tenzorus
        lemma_ids = self.tokenizer.encode(self.lemmas[idx], self.max_length)
        word_ids = self.tokenizer.encode(self.words[idx], self.max_length)
        return torch.tensor(lemma_ids, dtype=torch.long), torch.tensor(word_ids, dtype=torch.long)

# Funkcija, kas nodrošina, ka visas secības vienā datu porcijā ir vienāda garuma, aizpildot ar PAD tekstvienību
def collate_fn(batch):
    lemmas, targets = zip(*batch)
    pad_id = tokenizer.pad_token_id

    # Atrod garāko secību katrā gadījumā
    max_lemma_len = max(len(x) for x in lemmas)
    max_target_len = max(len(x) for x in targets)

    # Pagarina īsākās secības līdz noteiktajam garumam un atgriež rezultātu
    lemmas_padded = [torch.cat([x, torch.full((max_lemma_len - len(x),), pad_id, dtype=torch.long)]) for x in lemmas]
    targets_padded = [torch.cat([x, torch.full((max_target_len - len(x),), pad_id, dtype=torch.long)]) for x in targets]
    return torch.stack(lemmas_padded), torch.stack(targets_padded)

# Modeļa izveide

In [4]:
# Pozicionālā kodējuma izveide
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 32):
        super().__init__()
        # Izveido sākotnējo pozīciju tabulu (max_len x d_model), sākumā ar nullēm
        pe = torch.zeros(max_len, d_model)

        # Pozīciju vektora izveide
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Frekvenču skaitītājs
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model))

        # Pāra dimensijām sinuss
        pe[:, 0::2] = torch.sin(position * div_term)

        # Nepāra dimensijām kosinuss
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.cos(position * div_term[: pe[:, 1::2].shape[1]])
        else:
            pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe) # saglabā pe kā nemainīgu daļu no modeļa

    def forward(self, x):
        # Pievieno pozicionālo kodējumu reprezentācijām
        return x + self.pe[:, :x.size(1)]

# Kodētāja-dekodētāja modeļa klases definēšana
class CustomTransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=192, nhead=12, num_encoder_layers=2, num_decoder_layers=4, dim_feedforward=768, dropout=0.2, max_len=32):
        super(CustomTransformerModel, self).__init__()
        self.d_model = d_model

        # Pārvērš katru tekstvienības ID par {d_model} dimensiju vektoru
        self.embedding = nn.Embedding(vocab_size, d_model)

        # Slāņu normalizācija
        self.embed_ln = nn.LayerNorm(d_model)

        # Pozicionālā kodēšana tiek izmantota gan kodētājā, gan dekodētājā
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.pos_decoder = PositionalEncoding(d_model, max_len)

        # Inicializē transformera komponenti ar noteiktajiem parametriem
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, batch_first=True)

        # Lineārais slānis, lai no transformera izejas atgrieztu varbūtības katram tekstvienības ID
        self.fc_out = nn.Linear(d_model, vocab_size)

    def generate_square_subsequent_mask(self, sz):
        # Izveido masku, lai dekodētājs nespētu skatīties nākamās tekstvienības ģenerēšanas laikā
        mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
        return mask

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None, tgt_mask=None):
        # Iegūst reprezentācijas un pozicionālos kodējumus ievades un mērķa secībām
        src_emb = self.embed_ln(self.embedding(src)) * math.sqrt(self.d_model)
        src_emb = self.pos_encoder(src_emb)
        tgt_emb = self.embed_ln(self.embedding(tgt)) * math.sqrt(self.d_model)
        tgt_emb = self.pos_decoder(tgt_emb)

        # Izveido dekodētāja masku, lai nodrošinātu atbilstošu tekstvienību ģenerēšanu
        tgt_mask = self.generate_square_subsequent_mask(tgt_emb.size(1)).to(tgt_emb.device)

        # Iegūst izvadi
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask)

        # Projekcija uz vārdnīcas izmēra dimensiju izvada noteikšanai
        return self.fc_out(output)

# Iegūstam vārdnīcas izmēru un inicializējam modeli uz GPU, ja tas ir pieejams
vocab_size = tokenizer.vocab_size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomTransformerModel(vocab_size).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.01, ignore_index=tokenizer.pad_token_id) # PAD tokenam zaudējums netiek skaitīts

# Apmācības un vērtēšanas funkcijas

In [5]:
writer = SummaryWriter(log_dir="runs")
ckpt_dir = "checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)
global_step = 0
scaler = GradScaler()

# Apmācības cikls
def train_model(model, dataloader, optimizer, device, val_loader=None, log_every=10000):
    global global_step
    model.train()
    running_loss = 0.0 # kumulatīvais zaudējums pa datu porcijām

    # Iterācija caur datu porcijām
    for i, (src, tgt) in enumerate(tqdm(dataloader, desc="Train", leave=False)):
        # Datu porcijas sagatavošana
        src, tgt = src.to(device), tgt.to(device)
        tgt_in, tgt_gold = tgt[:, :-1], tgt[:, 1:]

        # Maskas PAD tekstvienībām
        src_mask = (src == tokenizer.pad_token_id)
        tgt_mask = (tgt_in == tokenizer.pad_token_id)

        optimizer.zero_grad()

        with autocast(device_type=device.type, enabled=(device.type=="cuda")):
             out = model(src, tgt_in, src_key_padding_mask=src_mask, tgt_key_padding_mask=tgt_mask)
             loss = criterion(out.reshape(-1, vocab_size), tgt_gold.reshape(-1))

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)

        if scheduler is not None:
            scheduler.step()

        scaler.update()

        # Palielina kumulatīvo zaudējumu un skaitītāju
        running_loss += loss.item()
        global_step += 1

        # Validācija ik pēc noteiktu soļu skaita
        if val_loader and global_step % log_every == 0:
            train_loss = running_loss / log_every
            val_loss = evaluate_loss(model, val_loader, device)

            # Fiksē rezultātus
            if writer is not None:
                writer.add_scalar("loss/train_step", train_loss, global_step)
                writer.add_scalar("loss/val_step", val_loss, global_step)

            print(f"step {global_step}: train = {train_loss:.4f} | val = {val_loss:.4f}")

            # Saglabā modeli ik pēc noteiktā soļu skaita kā pieturpunktu
            if ckpt_dir is not None:
                torch.save(model.state_dict(), f"{ckpt_dir}/step_{global_step}"f"_tr{train_loss:.3f}_val{val_loss:.3f}.pth")

            running_loss = 0.0 # atiestata kumulatīvo skaitītāju

    # Atgriež iterācijas vidējo apmācības zaudējumu
    return running_loss / len(dataloader)


# Apmācība un validācija
def train_and_validate(stage, train_loader, val_loader, optimizer, epochs, patience=0,
                       best_val_loss=float("inf"), prev_val_loss = 0, slow_count = 0, beam_k = 5, min_delta=0.005):
    use_early_stop = (patience > 0) # Agrā apstāšanās tikai tad, ja tā tiek iestatīta

    # Apmācības iterācija
    for ep in range(epochs):
        # Modeļa apmācība
        tr_loss = train_model(model, train_loader, optimizer, device, val_loader)

        # Zaudējuma aprēķināšana un modeļa novērtēšana
        val_loss = evaluate_loss(model, val_loader, device)

        if scheduler is not None:
            if isinstance(scheduler, ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()

        # Piefiksē mērījumu vērtības un izprintē tās
        if writer is not None:
            writer.add_scalars(f"{stage}/loss", {"train": tr_loss, "val": val_loss}, ep)
        print(f"{stage} {ep}/{epochs} | train_loss: {tr_loss:.4f} | val_loss: {val_loss:.4f}")

        # Ja netiek lietota agrā apstāšanās, turpina ar nākamajiem soļiem
        if not use_early_stop:
            continue

        improvement = best_val_loss - val_loss
        if improvement < min_delta:
            slow_count += 1
        else:
            slow_count = 0

        if improvement > 0:
            best_val_loss = val_loss

        # Ja 3× pēc kārtas zaudējums samazinās pārāk lēni, pārtrauc
        if slow_count >= patience:
            print("Notika agrā apstāšanās")
            break

# Funkcija, kas aprēķina vidējo zaudējumu validācijas kopai
def evaluate_loss(model, dataloader, device):
    model.eval()
    total_loss = torch.tensor(0.0, device=device)
    count = 0
    with torch.no_grad(), autocast(device_type=device.type, enabled=(device.type=="cuda")):
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            tgt_in  = tgt[:, :-1]
            tgt_gold= tgt[:,  1:]
            src_mask = (src == tokenizer.pad_token_id)
            tgt_mask = (tgt_in == tokenizer.pad_token_id)
            out = model(src, tgt_in, src_key_padding_mask=src_mask, tgt_key_padding_mask=tgt_mask)
            loss = criterion(out.view(-1, out.size(-1)), tgt_gold.reshape(-1))
            total_loss += loss
            count += 1
    return (total_loss / count).item()


# Kandidātu ģenerēšana un atbildēšana

In [6]:
# Atgriež beam search izveidotu sarakstu ar {num_beams} labākajām ģenerētajām virknēm teksta formā.
@torch.no_grad()
def beam_generate(model, src, max_len=32, num_beams=5):
    # Mainīgo sagatavošana
    device = src.device
    beams = [(torch.tensor([[tokenizer._vocab["[SOS]"]]], device=device), 0.0)]
    finished = []

    # Iterē max_length garumā
    for _ in range(max_len):
        new_beams = []

        for seq, score in beams:
            # Ja pēdējā tekstvienība ir EOS, secība ir pabeigta
            if seq[0, -1].item() == tokenizer._vocab["[EOS]"]:
                finished.append((seq, score))
                continue

            # Uztaisa masku dekoderim, lai neredz nākamās tekstvienības
            tgt_mask = model.generate_square_subsequent_mask(seq.size(1)).to(device)

            # Iegūst varbūtības tekstvienībām
            logits = model(src, seq,
                           src_key_padding_mask=(src == tokenizer.pad_token_id),
                           tgt_key_padding_mask=(seq == tokenizer.pad_token_id),
                           tgt_mask=tgt_mask)[:, -1]

            # Pārvērš logitus par log-varbūtībām
            logp = torch.log_softmax(logits, -1)[0]

            # Izvēlas top {num_beams} kandidātus no log-varbūtībām
            top_p, top_i = torch.topk(logp, num_beams)

            # Veido jaunu beamu katram top kandidātam
            for p, idx in zip(top_p, top_i):
                new_seq = torch.cat([seq, idx.view(1,1)], 1)
                new_beams.append((new_seq, score + p.item()))
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:num_beams]
        if not beams: break

    # Apvieno un sakārto pabeigtās secības
    finished += beams
    finished = sorted(finished, key=lambda x: x[1], reverse=True)[:num_beams]

    # Atgriež dekodētās secības kā tekstu, izņemot SOS/EOS/PAD tekstvienības
    return [
        tokenizer.decode(seq[0].tolist())
        for seq, _ in finished
    ]

# *Optuna* hiperparametru optimizācijas algoritms

In [7]:
# Iespējamās vērtības un to amplitūdas modeļa hiperparametriem un arhitektūrai
D_MODELS        = [64, 96, 128, 192, 256]
N_HEADS         = [2, 4, 6, 8, 12]
ENC_LAYERS      = [2, 4, 6]
DEC_LAYERS      = [2, 4, 6]
DIM_FF          = [2, 4, 6]
DROPOUT_BOUNDS  = (0.05, 0.5)
LR_BOUNDS       = (1e-6, 1e-3)
WDECAY_BOUNDS   = (1e-6, 1e-3)

def objective(trial: optuna.Trial):
    # Hiperparametru izloze
    d_model         = trial.suggest_categorical("d_model", D_MODELS)
    nhead           = trial.suggest_categorical("nhead", N_HEADS)
    num_enc_layers  = trial.suggest_categorical("num_encoder_layers", ENC_LAYERS)
    num_dec_layers  = trial.suggest_categorical("num_decoder_layers", DEC_LAYERS)
    dim_feedforward = trial.suggest_categorical("dim_ff", DIM_FF)
    dropout         = trial.suggest_float("dropout", *DROPOUT_BOUNDS)
    lr              = trial.suggest_float("lr", *LR_BOUNDS, log=True)
    weight_decay    = trial.suggest_float("weight_decay", *WDECAY_BOUNDS, log=True)

    # Nederīgo konfigurāciju atmešana (d_model jābūt dalāmam ar nhead)
    if d_model % nhead != 0:
        raise TrialPruned()

    # Modeļa inicializēšana ar izvēlētajiem hiperparametriem
    model = CustomTransformerModel(
        vocab_size=vocab_size,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_enc_layers,
        num_decoder_layers=num_dec_layers,
        dim_feedforward=dim_feedforward,
        dropout=dropout
    ).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Ātra apmācība un validācija uz mazas datu kopas
    train_model(model, small_train_loader, optimizer, device, val_loader=None)
    val_loss = evaluate_loss(model, small_val_loader, device)

    return val_loss

if run_optuna:
    # Nolasa pamatapmācības datni
    pretraining_df = pd.read_csv("LVK2022_filtrets.csv", sep=",", encoding="utf-8-sig")

    # Sadala pamatapmācības datu kopu 95% apmācībai un 5% validācijai
    pretraining_train_df, pretraining_val_df = train_test_split(pretraining_df, test_size=0.05)

    # Izveido dataset un dataloader mainīgos
    pretraining_train_dataset = WordPairDataset(pretraining_train_df, tokenizer)
    pretraining_val_dataset = WordPairDataset(pretraining_val_df, tokenizer)

    # Izmanto tikai 1/1000 daļu no pamatapmācības datu kopas ātrākai konfigurāciju meklēšanai
    train_subset_idx = list(range(len(pretraining_train_dataset) // 1000))
    val_subset_idx = list(range(len(pretraining_val_dataset) // 1000))
    small_train_loader = DataLoader(Subset(pretraining_train_dataset, train_subset_idx), batch_size=256, shuffle=True, collate_fn=collate_fn)
    small_val_loader = DataLoader(Subset(pretraining_val_dataset, val_subset_idx), batch_size=256, shuffle=False, collate_fn=collate_fn)

    # Optuna parametru sagatavošana
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", pruner=pruner, sampler=sampler)

    # Hiperparametru optimizācija 50 mēģinājumos
    study.optimize(objective, n_trials=50)

    print("Labākie parametri:", study.best_params)


# Pamatapmācība

In [8]:
if pretrain:
    # Nolasa pamatapmācības datni
    pretraining_df = pd.read_csv("LVK2022_filtrets.csv", sep=",", encoding="utf-8-sig")

    # Sadala pamatapmācības datu kopu 95% apmācībai un 5% validācijai
    pretraining_train_df, pretraining_val_df = train_test_split(pretraining_df, test_size=0.05)

    # Izveido dataset un dataloader mainīgos
    pretraining_train_dataset = WordPairDataset(pretraining_train_df, tokenizer)
    pretraining_val_dataset = WordPairDataset(pretraining_val_df, tokenizer)
    pretraining_train_loader = DataLoader(pretraining_train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn, num_workers=0, pin_memory=False)
    pretraining_val_loader = DataLoader(pretraining_val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=False)

    # Modeļa pamatapmācības cikls
    num_pretrain_epochs = 1
    optimizer = optim.AdamW(model.parameters(), lr=4e-4, weight_decay=1e-2)
    total_steps = len(pretraining_train_loader)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0 = total_steps // 2, T_mult = 1, eta_min = 1e-6)
    train_and_validate("Pretrained", pretraining_train_loader, pretraining_val_loader, optimizer, epochs=num_pretrain_epochs)

    # Saglabā pamatapmācītā modeļa svarus
    torch.save(model.state_dict(), "Pretrained.pth")

# Modeļa pielāgošana

In [9]:
if finetune:
    finetuning_files = [f"parveidojumi/parveidojums_{i}.csv" for i in range(1, 11)] # Sagatavo sarakstu ar pārveidojumu piemēru datnēm

    for idx, csv_path in enumerate(finetuning_files, start=1):
        # Nolasa pielāgošanas datu kopu
        finetuning_df = pd.read_csv(csv_path, sep=";", encoding="utf-8-sig")

        # Sadala pielāgošanas datu kopu 90% apmācībai un 10% validācijai
        finetuning_train_df, finetuning_val_df = train_test_split(finetuning_df, test_size=0.1)

        # Izveido dataset un dataloader mainīgos
        finetuning_train_dataset = WordPairDataset(finetuning_train_df, tokenizer)
        finetuning_val_dataset = WordPairDataset(finetuning_val_df, tokenizer)
        finetuning_train_loader = DataLoader(finetuning_train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
        finetuning_val_loader = DataLoader(finetuning_val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

        # Pārrakstām svarus izmantojot pamatapmācīto modeli
        model = CustomTransformerModel(vocab_size).to(device)
        model.load_state_dict(torch.load("Pretrained.pth", weights_only=True), strict=True)

        # Iesaldējam modeļa slāņus
        for n, p in model.named_parameters():
            p.requires_grad = False

        # Izmantojam LoRA modeļa pielāgošanai
        lora_cfg = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=[
              "decoder.layers.0.linear1", "decoder.layers.0.linear2", "decoder.layers.0.out_proj",
              "decoder.layers.1.linear1", "decoder.layers.1.linear2", "decoder.layers.1.out_proj",
              "decoder.layers.2.linear1", "decoder.layers.2.linear2", "decoder.layers.2.out_proj",
              "decoder.layers.3.linear1", "decoder.layers.3.linear2", "decoder.layers.3.out_proj", "fc_out"
            ]
        )
        model = get_peft_model(model, lora_cfg)

        # Parametri pielāgošanas procesam
        num_finetune_epochs = 50 # Maksimālais daudzums
        trainable = filter(lambda p: p.requires_grad, model.parameters())
        optimizer = torch.optim.AdamW(trainable, lr=5e-4)
        scheduler = None
        model.zero_grad()

        # Pielāgošana ar agrās apstāšanās kontroli
        train_and_validate(f"parveidojums_{idx}", finetuning_train_loader, finetuning_val_loader, optimizer, epochs=num_finetune_epochs, patience=4)

        # Saglabā PEFT adapteri ar unikālu nosaukumu
        save_dir = f"parveidojums_{idx}"
        model.save_pretrained(save_dir)
        print(f"Saglabāts adapteris: {save_dir}/\n")

parveidojums_1 0/50 | train_loss: 1.6452 | val_loss: 1.0830


parveidojums_1 1/50 | train_loss: 1.0240 | val_loss: 0.4909


parveidojums_1 2/50 | train_loss: 0.7075 | val_loss: 0.2839


parveidojums_1 3/50 | train_loss: 0.5555 | val_loss: 0.2651


parveidojums_1 4/50 | train_loss: 0.5364 | val_loss: 0.2547


...

parveidojums_10 44/50 | train_loss: 0.8038 | val_loss: 1.1990


parveidojums_10 45/50 | train_loss: 0.8456 | val_loss: 1.2018


parveidojums_10 46/50 | train_loss: 0.9507 | val_loss: 1.2079


parveidojums_10 47/50 | train_loss: 0.7784 | val_loss: 1.2104


parveidojums_10 48/50 | train_loss: 0.8068 | val_loss: 1.2167
Notika agrā apstāšanās
Saglabāts adapteris: parveidojums_10/



# Darbā definētās metodes izpilde

In [10]:
if experiment:
    # Vērtēšanas datu kopa formātā <lemma, lemmu grupas apzīmējums, sākotnējā semantiskā kategorija, mērķa semantiskā kategorija>
    lemmas = [
        ("veikt", "V", "Darīt", "Rezultāts"),
        ("zvaigzne", "V", "Priekšmets", "Ietver nosaukto"),
        ("pūst", "V", "Būt procesā", "Process"),
        ("riebt", "V", "Būt stāvoklī", "Stāvoklis"),
        ("medicīna", "V", "Abstrakts nojēgums", "Saistīts ar nosaukto"),
        ("skriet", "V", "Darīt", "Darītājs (dzīvs)"),
        ("ideja", "V", "Abstrakts nojēgums", "Saistīts ar nosaukto"),

        ("nest", "I", "Darīt", "Darbība"),
        ("iet", "I", "Būt procesā", "Process"),
        ("ir", "I", "Būt stāvoklī", "Stāvoklis"),

        ("atklusināt", "J", "Darīt", "Instruments"),
        ("sejauts", "J", "Priekšmets", "Ietver nosaukto"),
        ("telpiskot", "J", "Būt procesā", "Process"),
        ("sliecināt", "J", "Būt stāvoklī", "Stāvoklis"),
        ("aizstājeklis", "J", "Abstrakts nojēgums", "Saistīts ar nosaukto"),

        ("tērbstīt", "N", "Darīt", "Vieta (lietv)"),
        ("plakstule", "N", "Priekšmets", "Ietver nosaukto"),
        ("glimžēt", "N", "Būt procesā", "Process"),
        ("skurpelīgs", "N", "Būt stāvoklī", "Stāvoklis"),
        ("zvilgsme", "N", "Abstrakts nojēgums", "Saistīts ar nosaukto")
    ]

    # Iespējamie pārveidojumu veidi
    POSSIBLE_TRANSFORMATIONS = [
        ("Darīt", "Darbība"),
        ("Darīt", "Rezultāts"),
        ("Darīt", "Darītājs (dzīvs)"),
        ("Priekšmets", "Ietver nosaukto"),
        ("Būt procesā", "Process"),
        ("Būt stāvoklī", "Stāvoklis"),
        ("Abstrakts nojēgums", "Saistīts ar nosaukto"),
        ("Darīt", "Vieta (lietv)"),
        ("Darīt", "Cits"),
        ("Darīt", "Instruments")
    ]

    _model_cache = {}

    # Funkcija, kas automātiski izvēlas modeli atbilstoši pārveidojumam
    def get_finetuned_model(semantic_category_1, semantic_category_2):
        try:
            idx = POSSIBLE_TRANSFORMATIONS.index((semantic_category_1, semantic_category_2))
        except ValueError:
            return None

        ckpt = f"parveidojums_{idx+1}"

        # Ja modelis jau nav saglabāts kešatmiņā, tad saglabā, lai vēlāk atkārtoti to izmantotu
        if ckpt not in _model_cache:
            # Ielādē bāzes svarus
            base = CustomTransformerModel(vocab_size).to(device)
            base.load_state_dict(torch.load("Pretrained.pth", map_location=device, weights_only=True), strict=True)

            # Pieliek PEFT adapteri, kas glabājas mapē `ckpt`
            m = PeftModel.from_pretrained(base, ckpt, local_files_only=True, is_trainable=False).to(device)
            m.eval()
            _model_cache[ckpt] = m
        return _model_cache[ckpt]

    # Funkcija, kas validē kandidātu, izmantojot korpusa API
    def validate_word(word):
        encoded_word = urllib.parse.quote(word)
        url = ("https://nosketch.korpuss.lv/bonito/run.cgi/view?"
              "corpname=CommonCrawl&format=json&pagesize=2&fromp=0&attrs=word&"
              "ctxattrs=word%2Ctag&kwicleftctx=5%23&kwicrightctx=5%23&async=0&"
              f"q=q[lc%3D%22{encoded_word}%22+|+lemma_lc%3D%22{encoded_word}%22]")

        try:
            r = requests.get(url, timeout=10)
            if r.status_code == 200:
                freq = r.json().get("fullsize", 0)
                if freq == 0:
                    typ = "F" # nav sastopams
                elif 1 <= freq <= 5:
                    typ = "M" # maz sastopams - nepieciešama manuāla pārbaude
                else:
                    typ = "T" # daudz sastopams
                return typ, freq
            return "validation_error", None
        except Exception as e:
            print(f"Validācijas kļūda vārdam '{word}': {e}")
            return "validation_error", None

    results = []

    # Iterē caur katru derivējamo lemmu
    for lemma in lemmas:
        word = lemma[0]
        semantic_category_1 = lemma[2]
        semantic_category_2 = lemma[3]

        model = get_finetuned_model(semantic_category_1, semantic_category_2)

        if model is None:
            print(f"Nav modeļa pārveidojumam ({semantic_category_1}→{semantic_category_2})")
            continue

        src_tok = torch.tensor([tokenizer.encode(word)]).to(device)
        response = beam_generate(model, src_tok, num_beams=5)
        print(f"{lemma[0]}, {lemma[2]}, {lemma[3]}, {response}")

        # Validē katru kandidātu pret korpusu kolekciju un pievieno rezultātu sarakstam
        for cand in response:
            typ, freq = validate_word(cand)
            results.append({
                "Lemma": word,
                "Semantiskā_kat_1": semantic_category_1,
                "Kandidāts": cand,
                "Semantiskā_kat_2": semantic_category_2,
                "Biežums": freq,
                "Tips": typ,
                "Grupa": lemma[1]
            })

    # Saglabā rezultātus TSV failā, kas ir formātā '<metode/modelis>.tsv'
    output_tsv = f"special_model_results.tsv"

    with open(output_tsv, "w", newline="", encoding="utf-8-sig") as tsvfile:
        writer = csv.writer(tsvfile, delimiter="\t")
        header = ["Lemma", "Semantiskā_kat_1", "Kandidāts", "Semantiskā_kat_2", "Biežums", "Tips", "Grupa"]
        writer.writerow(header)

        for row in results:
            out_row = [row["Lemma"], row["Semantiskā_kat_1"], row["Kandidāts"], row["Semantiskā_kat_2"],
                      row["Biežums"], row["Tips"], row["Grupa"]]
            writer.writerow(out_row)

    print(f"Rezultāti saglabāti TSV failā: {output_tsv}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


veikt, Darīt, Rezultāts, ['veikums', 'veicums', 'veiks', 'veikba', 'veiciekums']
zvaigzne, Priekšmets, Ietver nosaukto, ['zvaigžņots', 'zvaigžņains', 'zvaigznins', 'zvaigznots', 'zvaigznains']
pūst, Būt procesā, Process, ['pūšana', 'pūšanās', 'pūsta', 'pūšanā', 'pūšanām']
riebt, Būt stāvoklī, Stāvoklis, ['riebšana', 'rieba', 'riebta', 'riebdana', 'riebšanas']
medicīna, Abstrakts nojēgums, Saistīts ar nosaukto, ['medicīns', 'medicīnis', 'medicīnsks', 'medicīnisks', 'medicīnējs']
skriet, Darīt, Darītājs (dzīvs), ['skrienējs', 'skrienētājs', 'skrienis', 'skrietājs', 'skriecējs']
ideja, Abstrakts nojēgums, Saistīts ar nosaukto, ['ides', 'idesks', 'idejs', 'idejis', 'idejēs']
nest, Darīt, Darbība, ['nesīšana', 'nesta', 'nesšana', 'nestu', 'nests']
iet, Būt procesā, Process, ['iešana', 'ieta', 'iešanā', 'iešanās', 'iets']
ir, Būt stāvoklī, Stāvoklis, ['ires', 'iru', 'irā', 'ira', 'iri']
atklusināt, Darīt, Instruments, ['atklusinātājs', 'atklusinājs', 'atklusināts', 'atklusinātājjs', 'atklusi

# Pamattrenētā modeļa vispārīga pārbaude

In [18]:
# Lietotāja ievade
user_input = input("Ievadi lemmu: ")

model = CustomTransformerModel(vocab_size).to(device)
model.load_state_dict(torch.load("Pretrained.pth", weights_only=True), strict=True)

# Ievades secība dalīšana tekstvienībās
src_tok = torch.tensor([tokenizer.encode(user_input)]).to(device)

# Ģenerējam 5 kandidātus
print(beam_generate(model, src_tok, num_beams=5))

Ievadi lemmu: rakstīt
['raksta', 'rakstīja', 'rakstīta', 'rakstītas', 'rakstīts']
